In [ ]:
# [2]
import carla
import math
import random
import time
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
# [3]
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(carla.Location(x=-4, z=2.5), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
# [6]
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# save to png sequence
# camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [ ]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [ ]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))}

camera.listen(lambda image: camera_callback(image, camera_data))

In [ ]:
vehicle.set_autopilot(True)

In [ ]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['image'])
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()